# 3D OSZTALYOZAS

In [10]:
import os
import numpy as np
import keras
import trimesh
import tqdm
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from pointnet.dataset import ModelNetDataset, gen_modelnet_id
from torch.utils.data import DataLoader
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.neighbors import NearestNeighbors
import numpy as np
from torch_geometric.utils import from_scipy_sparse_matrix
from torch.utils.data import Dataset,DataLoader
from learning3d.models import DGCNN
from pointnet.model import PointNetCls


In [ ]:
DATA_DIR = keras.utils.get_file(
    "modelnet.zip",
    "http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip",
    extract=True,
)
DATA_DIR = os.path.join(os.path.dirname(DATA_DIR), "ModelNet10")

In [ ]:
print(DATA_DIR)

## Feature keszites az SVM-hez, RF-hez

In [3]:
def bounding_box_dimension(mesh):
    return mesh.bounding_box.extents
def surface_area(mesh):
    return mesh.area
def volume(mesh):
    return mesh.volume
def compactness(mesh):
    return mesh.volume**2 / mesh.area**3
def eccentricity(mesh):
    eigenvalues = np.linalg.eigvalsh(np.cov(mesh.vertices.T))
    return np.sqrt(eigenvalues[-1] / eigenvalues[0])
def genus(mesh):
    return mesh.euler_number // 2
def euler_characteristic(mesh):
    return len(mesh.vertices) - len(mesh.edges) + len(mesh.faces)
def num_connected_components(mesh):
    return len(mesh.split())

In [3]:
d_path = "C:/Users/daneb/.keras/datasets/modelnet_extracted/ModelNet10"

In [6]:
def extract_features(mesh):
    features = []
    features.extend(bounding_box_dimension(mesh))
    features.append(surface_area(mesh))
    features.append(volume(mesh))
    features.append(compactness(mesh))
    features.append(eccentricity(mesh))
    features.append(genus(mesh))
    features.append(euler_characteristic(mesh))
    features.append(num_connected_components(mesh))
    return features

output_csv = "modelnet10_features.csv"
with open(output_csv, 'w') as f:
    f.write("label,bbd_x,bbd_y,bbd_z,s_area,volume,compact,eccent,genus,euler,num_conn\n")

for class_name in tqdm.tqdm([d for d in os.listdir(d_path) if os.path.isdir(os.path.join(d_path, d))], desc="Classes"):
    class_path = os.path.join(d_path, class_name)
    if not os.path.isdir(class_path):
        continue  
    for split in ['train']:
        split_path = os.path.join(class_path, split)
        mesh_files = [file for file in os.listdir(split_path) if file.endswith(('.off'))]
        for file in tqdm.tqdm(mesh_files, desc=f"{class_name}/{split}", leave=False):
            mesh_path = os.path.join(split_path, file)
            mesh = trimesh.load(mesh_path)
            features = extract_features(mesh)
            with open(output_csv, 'a') as f:
                f.write(f"{class_name},{','.join(map(str, features))}\n")


Classes:   0%|          | 0/10 [00:00<?, ?it/s]

Classes: 100%|██████████| 10/10 [05:03<00:00, 30.33s/it]


In [7]:
mesh_features_df = pd.read_csv("modelnet10_features.csv")

In [8]:
mesh_features_df.head()

,label,bbd_x,bbd_y,bbd_z,s_area,volume,compact,eccent,genus,euler,num_conn
0,bathtub,32.256600,54.000500,40.211500,32701.112088,1.531873e+00,6.710543e-14,1.655826,794,-6078,0
1,bathtub,35.243882,59.489766,22.750000,17020.348330,-2.425319e-12,1.192978e-36,1.825356,167,-1184,0
2,bathtub,27.559100,66.929100,21.535390,12562.333554,2.142343e+04,2.315086e-04,3.291839,2038,-32221,4
3,bathtub,61.000040,61.000000,36.574800,18972.581747,3.620864e+04,1.919752e-04,2.749974,268,-5531,0
4,bathtub,31.647400,61.016600,29.846396,7520.084685,1.447286e+03,4.925390e-06,2.456193,-21,-46055,1


In [9]:
print(mesh_features_df.isnull().sum())

label       0
bbd_x       0
bbd_y       0
bbd_z       0
s_area      0
volume      0
compact     0
eccent      0
genus       0
euler       0
num_conn    0
dtype: int64


In [10]:
X = mesh_features_df.drop(columns=['label']) 
y = mesh_features_df['label']

In [11]:
print(y.value_counts())

label
chair          889
sofa           680
bed            515
monitor        465
table          392
toilet         344
desk           200
dresser        200
night_stand    200
bathtub        106
Name: count, dtype: int64


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

svm = make_pipeline(StandardScaler(), SVC(kernel='rbf', random_state=42))
svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)

print("Classification Report:")
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")


Classification Report:
              precision    recall  f1-score   support

     bathtub       0.00      0.00      0.00        32
         bed       0.48      0.08      0.14       155
       chair       0.41      0.93      0.57       267
        desk       0.00      0.00      0.00        60
     dresser       0.48      0.17      0.25        60
     monitor       0.37      0.19      0.25       139
 night_stand       0.67      0.03      0.06        60
        sofa       0.36      0.82      0.50       204
       table       0.00      0.00      0.00       118
      toilet       0.22      0.02      0.04       103

    accuracy                           0.39      1198
   macro avg       0.30      0.22      0.18      1198
weighted avg       0.33      0.39      0.28      1198

Accuracy: 39.07%


c:\Users\daneb\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\daneb\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\daneb\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")

Classification Report:
              precision    recall  f1-score   support

     bathtub       0.57      0.38      0.45        32
         bed       0.73      0.68      0.70       155
       chair       0.88      0.93      0.91       267
        desk       0.53      0.35      0.42        60
     dresser       0.85      0.77      0.81        60
     monitor       0.78      0.79      0.79       139
 night_stand       0.73      0.72      0.72        60
        sofa       0.67      0.86      0.75       204
       table       0.73      0.64      0.68       118
      toilet       0.89      0.79      0.84       103

    accuracy                           0.77      1198
   macro avg       0.74      0.69      0.71      1198
weighted avg       0.76      0.77      0.76      1198

Accuracy: 76.63%


## PointNet
[Forras - 1](https://arxiv.org/pdf/1612.00593)
[Forras - 2](https://keras.io/examples/vision/pointnet/)
[Forras - 3](https://github.com/fxia22/pointnet.pytorch)

In [ ]:
with open(os.path.join(d_path, 'train.txt'), 'w') as train_f, \
     open(os.path.join(d_path, 'test.txt'), 'w') as test_f:
    for cls in os.listdir(d_path):
        cls_path = os.path.join(d_path, cls)
        if os.path.isdir(cls_path):  
            for split in ['train', 'test']:  
                split_path = os.path.join(cls_path, split)
                if os.path.isdir(split_path):  
                    for file in os.listdir(split_path):
                        if file.endswith('.ply'):  
                            line = f"{cls}/{split}/{file}\n"
                            if split == 'train':
                                train_f.write(line)
                            else:
                                test_f.write(line)

print("Generated train.txt and test.txt successfully.")

Generated train.txt and test.txt successfully.


In [5]:
os.chdir('pointnet.pytorch')

In [6]:
d_path = "C:/Users/daneb/.keras/datasets/modelnet_extracted/ModelNet10"


In [7]:
gen_modelnet_id(d_path)

train_dataset = ModelNetDataset(root=d_path, split='train', npoints=1024, data_augmentation=True)

test_dataset = ModelNetDataset(root=d_path, split='test', npoints=1024, data_augmentation=False)


{'bathtub': 0, 'bed': 1, 'chair': 2, 'desk': 3, 'dresser': 4, 'monitor': 5, 'night_stand': 6, 'sofa': 7, 'table': 8, 'toilet': 9}
{'bathtub': 0, 'bed': 1, 'chair': 2, 'desk': 3, 'dresser': 4, 'monitor': 5, 'night_stand': 6, 'sofa': 7, 'table': 8, 'toilet': 9}


In [ ]:
def convert_off_to_ply(off_path, ply_path):
    mesh = trimesh.load(off_path)
    mesh.export(ply_path)

for root, dirs, files in os.walk(d_path):
    for file in files:
        if file.endswith('.off'):
            off_file = os.path.join(root, file)
            ply_file = off_file.replace('.off', '.ply')
            convert_off_to_ply(off_file, ply_file)

In [8]:
train_dataset = ModelNetDataset(root=d_path, split='train', npoints=1024, data_augmentation=True)
test_dataset = ModelNetDataset(root=d_path, split='test', npoints=1024, data_augmentation=False)

{'bathtub': 0, 'bed': 1, 'chair': 2, 'desk': 3, 'dresser': 4, 'monitor': 5, 'night_stand': 6, 'sofa': 7, 'table': 8, 'toilet': 9}
{'bathtub': 0, 'bed': 1, 'chair': 2, 'desk': 3, 'dresser': 4, 'monitor': 5, 'night_stand': 6, 'sofa': 7, 'table': 8, 'toilet': 9}


In [16]:
batch_size = 32
npoints = 1024
epochs = 13
learning_rate = 0.0005 

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)


In [15]:
print("Contents of ModelNet10:")
for item in os.listdir(d_path):
    print(item)

Contents of ModelNet10:
.DS_Store
bathtub
bed
chair
desk
dresser
monitor
night_stand
README.txt
sofa
table
test.txt
toilet
train.txt


In [17]:
device = torch.device('cpu')  

classifier = PointNetCls(k=10)
classifier.to(device)  

optimizer = optim.Adam(classifier.parameters(), lr=learning_rate,weight_decay=1e-3)


for epoch in range(epochs):
    classifier.train()  
    running_loss = 0.0
    correct = 0
    total = 0
    
    for i, (points, labels) in enumerate(train_loader):
        points, labels = points.float().to(device), labels.long().to(device)
        
        
        points = points.transpose(2, 1)  

        labels = labels.squeeze()  

        optimizer.zero_grad()

        outputs, *_ = classifier(points) 

        loss = F.cross_entropy(outputs, labels)  
        
        loss.backward()
        
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

    classifier.eval()  
    correct = 0
    total = 0
    with torch.no_grad():
        for points, labels in test_loader:
            points, labels = points.float().to(device), labels.long().to(device)
            points = points.transpose(2, 1)  

            labels = labels.squeeze()  

            outputs, *_ = classifier(points)  
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_accuracy = 100 * correct / total
    print(f"Test Accuracy after Epoch [{epoch+1}/{epochs}]: {test_accuracy:.2f}%")

Epoch [1/13], Loss: 1.3622, Accuracy: 56.80%
Test Accuracy after Epoch [1/13]: 59.36%
Epoch [2/13], Loss: 1.0038, Accuracy: 67.28%
Test Accuracy after Epoch [2/13]: 66.63%
Epoch [3/13], Loss: 0.8712, Accuracy: 71.03%
Test Accuracy after Epoch [3/13]: 58.48%
Epoch [4/13], Loss: 0.8027, Accuracy: 73.99%
Test Accuracy after Epoch [4/13]: 69.16%
Epoch [5/13], Loss: 0.7524, Accuracy: 74.79%
Test Accuracy after Epoch [5/13]: 70.15%
Epoch [6/13], Loss: 0.7408, Accuracy: 75.12%
Test Accuracy after Epoch [6/13]: 69.05%
Epoch [7/13], Loss: 0.7190, Accuracy: 76.05%
Test Accuracy after Epoch [7/13]: 73.02%
Epoch [8/13], Loss: 0.6788, Accuracy: 76.92%
Test Accuracy after Epoch [8/13]: 68.28%
Epoch [9/13], Loss: 0.6511, Accuracy: 78.03%
Test Accuracy after Epoch [9/13]: 66.52%
Epoch [10/13], Loss: 0.6399, Accuracy: 78.55%
Test Accuracy after Epoch [10/13]: 69.71%
Epoch [11/13], Loss: 0.6436, Accuracy: 78.70%
Test Accuracy after Epoch [11/13]: 70.93%
Epoch [12/13], Loss: 0.6228, Accuracy: 79.35%
Test

## DGCNN

In [ ]:
class ModelNet10Dataset(Dataset):
    def __init__(self, d_path, split='train', num_points=1024, txt_file=None):
        self.dataset_path = d_path
        self.split = split
        self.num_points = num_points
        self.txt_file = txt_file  
        self.mesh_paths = []
        self.labels = []
        
        self.classes = sorted([cls for cls in os.listdir(d_path) if os.path.isdir(os.path.join(d_path, cls))])
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}

        
        self.load_mesh_paths_and_labels()

    def load_mesh_paths_and_labels(self):
        if not os.path.exists(self.txt_file):
            raise FileNotFoundError(f"File {self.txt_file} does not exist.")
        
        with open(self.txt_file, 'r') as f:
            lines = f.readlines()
            for line in lines:
                mesh_path = line.strip()
                if not mesh_path:
                    print(f"Skipping empty line.")
                    continue
                
                class_name = mesh_path.split('/')[0]
                
                if class_name not in self.class_to_idx:
                    print(f"Skipping unknown class: {class_name}")
                    continue
                
                label = self.class_to_idx[class_name]
                
                full_path = os.path.join(self.dataset_path, mesh_path)
                if not os.path.exists(full_path):
                    print(f"Warning: File {full_path} does not exist.")
                    continue
                
                self.mesh_paths.append(full_path)
                self.labels.append(label)

        if len(self.mesh_paths) == 0:
            raise ValueError(f"No valid data found in {self.txt_file}.")

    def __len__(self):
        return len(self.mesh_paths)

    def __getitem__(self, idx):
        mesh_path = self.mesh_paths[idx]
        pointcloud = self.load_pointcloud(mesh_path)
        label = self.labels[idx]
        pointcloud = torch.tensor(pointcloud, dtype=torch.float32).transpose(0, 1)  
        return pointcloud, torch.tensor(label, dtype=torch.long)


    def load_pointcloud(self, path):
        mesh = trimesh.load_mesh(path)
        pointcloud, _ = trimesh.sample.sample_surface(mesh, self.num_points)
        return pointcloud


In [ ]:
d_path = "C:/Users/daneb/.keras/datasets/modelnet_extracted/ModelNet10"
train_txt_file = "C:/Users/daneb/.keras/datasets/modelnet_extracted/ModelNet10/train.txt"
test_txt_file = "C:/Users/daneb/.keras/datasets/modelnet_extracted/ModelNet10/test.txt"

train_dataset_dgcnn = ModelNet10Dataset(d_path, split='train', num_points=1024, txt_file=train_txt_file)
test_dataset_dgcnn = ModelNet10Dataset(d_path, split='test', num_points=1024, txt_file=test_txt_file)

train_loader = DataLoader(train_dataset_dgcnn, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset_dgcnn, batch_size=32, shuffle=False)

pointcloud, label = train_dataset_dgcnn[0]
print(pointcloud.shape)  
print(label) 


torch.Size([3, 1024])
tensor(0)


In [ ]:
class DGCNN_Classifier(torch.nn.Module):
    def __init__(self, num_classes=10, emb_dims=1024):
        super(DGCNN_Classifier, self).__init__()
        self.dgcnn = DGCNN(emb_dims=emb_dims)
        self.fc1 = torch.nn.Linear(emb_dims, 512)
        self.bn1 = torch.nn.BatchNorm1d(512)
        self.fc2 = torch.nn.Linear(512, 256)
        self.bn2 = torch.nn.BatchNorm1d(256)
        self.dp1 = torch.nn.Dropout(p=0.5)
        self.fc3 = torch.nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.dgcnn(x)  
        x = x.max(dim=2)[0]  
        x = F.relu(self.bn1(self.fc1(x)))
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dp1(x)
        x = self.fc3(x)
        return x

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DGCNN_Classifier(num_classes=10).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

for epoch in range(20):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for points, labels in train_loader:
        points, labels = points.to(device), labels.to(device)
        points = points.permute(0, 2, 1)  
        
        optimizer.zero_grad()
        outputs = model(points)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()

        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    avg_loss = total_loss / len(train_loader)
    acc = 100.0 * correct / total
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}, Accuracy: {acc:.2f}%")


Epoch 1, Loss: 1.7173, Accuracy: 43.55%
Epoch 2, Loss: 0.8947, Accuracy: 70.66%
Epoch 3, Loss: 0.5982, Accuracy: 80.83%
Epoch 4, Loss: 0.4671, Accuracy: 85.22%
Epoch 5, Loss: 0.4014, Accuracy: 86.75%
Epoch 6, Loss: 0.3661, Accuracy: 88.27%
Epoch 7, Loss: 0.3292, Accuracy: 89.48%
Epoch 8, Loss: 0.3131, Accuracy: 89.55%
Epoch 9, Loss: 0.3237, Accuracy: 89.55%
Epoch 10, Loss: 0.2814, Accuracy: 90.43%
Epoch 11, Loss: 0.2842, Accuracy: 90.45%
Epoch 12, Loss: 0.2575, Accuracy: 91.86%
Epoch 13, Loss: 0.2457, Accuracy: 92.18%
Epoch 14, Loss: 0.2228, Accuracy: 92.33%
Epoch 15, Loss: 0.2236, Accuracy: 92.53%
Epoch 16, Loss: 0.1936, Accuracy: 93.56%
Epoch 17, Loss: 0.2212, Accuracy: 92.63%
Epoch 18, Loss: 0.2100, Accuracy: 92.71%
Epoch 19, Loss: 0.2069, Accuracy: 93.16%
Epoch 20, Loss: 0.1892, Accuracy: 93.76%


In [23]:
torch.save(model.state_dict(), "dgcnn_model_weights.pth")

## ZARAS, OSSZEGZES
Hasznalt adathalmaz: [ModelNet10 - Princeton](http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip")

Erintentett modszerek:
- SVM
- RandomForest
- PointNet
- DGCNN

| Modszer/modell | Pontossag | Bemenet                          |
|----------------|-----------|----------------------------------|
| SVM            |     39.07%      | Topologiai/Geometriai leirok     |
| RandomForest   |     76.63%      | Topologiai/Geometriai leirok     |
| PointNet       |     79.35%      | Point Cloud                      |
| DGCNN          |     93.56%| Point Cloud, de graf generalodik |
